<a href="https://colab.research.google.com/github/ryanbinny/Main-Project/blob/main/LLM_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers peft accelerate bitsandbytes datasets trl
# ! for executing in shell mode
# -q is quiet mode-suppresses most of the logs so that the output is cleaner
#transformers-hf library for traing ML and LLM models
#datasets-hf library for downloading and processing datasets
#accelerate-helps run models on different hardwares like cpu, gpu, etc.
#bitsandbytes-used for 8 we bit and 4bit quantization(reduce the precision of the numbers used to store the model's weights) => larger models will be able to run on smaller GPU's
#peft-(Parameter Efficient Fine Tuning)-used for fine-tuning techniques like Lora,QLora => dont need to retrain the entire model
#trl-(Transformer Reinforcement Learning)-hf library for training transformer models with reinforcement learning (eg. RLHF-reinforcement Learning from Human Feedback)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 841.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#AutoTokenizer- a class that automatically detects the correct tokenizer for a given model(text->numbers->back to text)
#AutoModelForCausalLM-loads a Causal Language Model (CLM) based on the model name automatically-these models are trained to predict the next word in a sequence (text generation model)

model_name="jan-hq/LlamaCorn-1.1B-Chat"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained("LlamaCorn_model")
tokenizer.save_pretrained("LlamaCorn_model")
# save the two models locally as LlamaCorn model

print("✅ Model and tokenizer saved")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

✅ Model and tokenizer saved


In [ ]:
pip uninstall fitz -y


Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Successfully uninstalled fitz-0.0.1.dev2


In [ ]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 26.9 MB/s eta 0:00:00


In [ ]:
import os
# helps to interact with file paths, environment variables, process control
import json
import fitz
#Comes from PyMuPDF-used for reading, writing and editing pdfs
import textwrap
#used for formatting text to a certain width
import torch
#used for tensors, DL models and gpu acceleration
from transformers import AutoTokenizer,AutoModelForCausalLM, pipeline

def extract_text_from_pdf(pdf_path):
  doc = fitz.open(pdf_path)
  text=""
  for page in doc:
    text += page.get_text()
  return text

def extract_texts_from_folder(folder_path):
  all_text=""
  for filename in os.listdir(folder_path):
    if filename.lower().endswith(".pdf"):
      print(f"Extracting text from : {filename}")
      file_path = os.path.join(folder_path, filename)
      pdf_text = extract_text_from_pdf(file_path)
      #we are calling this fn again to extract text from first a single pdf then the other pdfs
      all_text +="\n" + pdf_text
  return all_text.strip()

def split_into_chunks(all_text, chunk_size = 500):
  return textwrap.wrap(all_text, chunk_size)

def build_prompt(context):
  return f"""You are a compassioante counselling assistant trained to help adolescents. Based on the following passage, generate an emotionally understanding question and a kind supportive answer:
  Passage:{context}
  Your answer should show empathy and care.
  Question:"""

def extract_qa_pairs(all_text):
  all_text = all_text.strip()
  question = answer = None
  if "Question:" in all_text:
    parts = all_text.split("Question:",1)[-1]
    #splits the text into two, -1 means we are taking the last element of the list which split() has returned, so the part after qn
    if "Answer:" in parts:
      question, answer = parts.split("Answer:",1)
      #tuple unpacking- qn and ans will now become separate
    else:
      question = parts.strip()
  if question:
    question = question.strip().replace("\n","")
    #we use the replace so that no inline characters between strings will be present
  if answer:
    answer = answer.strip().replace("\n","")
  return question, answer

if __name__ == "__main__":
  folder_path = "/content/drive/MyDrive/LLMCounsellingChatbot/data/pdfs"
  print("Extracting and chunking texts")
  combined_text = extract_texts_from_folder(folder_path)
  chunks = split_into_chunks(combined_text)
  print(f"Split into {len(chunks)} chunks")
  print("Loading model...")
  model_path = "/content/LlamaCorn_model"
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  model = AutoModelForCausalLM.from_pretrained(model_path)

  qa_generator = pipeline(
      "text-generation",
      model=model,
      tokenizer = tokenizer,
      device=0 if torch.cuda.is_available() else -1)
  # we are creating a pipeline ie. an end-to-end workflow, we are telling it to perform task generation if device=0 ie. if gpu is available else use cpu

  print("Generating Q&A pairs")
  qa_pairs = []
  for i, chunk in enumerate(chunks[:50]):
    #enumerate to automatically identify the num of chunks so that value of i ie. the index no. will also have the same value
    print(f"Generating for chunk {i+1} / {min(len(chunks), 50)}")
    prompt = build_prompt(chunk)
    output = qa_generator(prompt, max_new_tokens = 100, do_sample = True, temperature = 0.7)[0]["generated_text"]
    #[0] means the first part from the output is only stored from the generated text pipeline
    question, answer = extract_qa_pairs(output)
    if question and answer:
      qa_pairs.append({"question": question, "answer": answer})
    else:
      print(f"Skipped chunk {i+1} : Could not extract clean Q&A")
    output_path = "/content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json"
    with open(output_path, "w", encoding = "utf-8") as f:
      json.dump(qa_pairs, f, ensure_ascii=False, indent = 2)
      #indent means use 2 spaces for each indent level
    print(f"✅ Saved {len(qa_pairs)} Q&A pairs to {output_path}")

Extracting and chunking texts
Extracting text from : LernerSteinberg-eds-Handbook-of-Adolescent-Psychology.pdf
Extracting text from : MRS. MENKITI 2009 adolesencent  psychology.pdf
Extracting text from : Manual on Youth and Adolescents Counselling (Eng).pdf
Extracting text from : develop.pdf
Extracting text from : 9781260058789.pdf
Extracting text from : Adolescent-Development-Textbook.pdf
Extracting text from : HandbookChild.Adol.Clin.Psycho.Context.approach.pdf
Split into 16131 chunks
Loading model...


Device set to use cuda:0


Generating Q&A pairs
Generating for chunk 1 / 50
✅ Saved 1 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 2 / 50
✅ Saved 2 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 3 / 50
Skipped chunk 3 : Could not extract clean Q&A
✅ Saved 2 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 4 / 50
✅ Saved 3 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 5 / 50
✅ Saved 4 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 6 / 50
✅ Saved 5 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 7 / 50
✅ Saved 6 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 8 / 50
✅ Saved 7 Q&A pairs to /content/drive/MyDriv

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Skipped chunk 10 : Could not extract clean Q&A
✅ Saved 8 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 11 / 50
✅ Saved 9 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 12 / 50
✅ Saved 10 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 13 / 50
Skipped chunk 13 : Could not extract clean Q&A
✅ Saved 10 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 14 / 50
Skipped chunk 14 : Could not extract clean Q&A
✅ Saved 10 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 15 / 50
Skipped chunk 15 : Could not extract clean Q&A
✅ Saved 10 Q&A pairs to /content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json
Generating for chunk 16 / 50
✅ Saved 11 Q&A pairs to /content/drive/MyDrive/LLMCounsellingCha

In [ ]:
import json
import re

input_path = "/content/drive/MyDrive/LLMCounsellingChatbot/data/cleaned_qa_pairs.json"
output_path = "/content/drive/MyDrive/LLMCounsellingChatbot/data/finetune_ready_data.json"

with open(input_path, "r", encoding="utf-8") as f:
  data = json.load(f)

cleaned_data = []

for item in data:
  q=item.get("question","").strip()
  #"" is the value to be returned if question is not found
  a=item.get("answer","").strip()

  q = re.sub(r"<\|.*?\|>","",q).strip()
  # we are using sub method-find characters that match netween <\| and \|>- .*? is a non greedy search technique to match as little characters as possible before the first \|>
  a = re.sub(r"<\|.*?\|>","",a).strip()
  if q and a:
    formatted = {
        "messages":[
            {"role":"user","content":q},
            #the users message content
            {"role":"assistant","content":a}
            #the chatbot reply content
        ]
    }
    cleaned_data.append(formatted)

print(f"Total raw items loaded: {len(data)}")
print(f"Cleaned Q&A pairs: {len(cleaned_data)}")

with open(output_path, "w", encoding="utf-8") as f:
  json.dump(cleaned_data, f, indent=2, ensure_ascii=False)
print(f"File saved to : {output_path}")


Total raw items loaded: 27
Cleaned Q&A pairs: 27
File saved to : /content/drive/MyDrive/LLMCounsellingChatbot/data/finetune_ready_data.json


In [ ]:
import json

with open("/content/drive/MyDrive/LLMCounsellingChatbot/data/finetune_ready_data.json", "r") as f:
  data = json.load(f)

from datasets import Dataset
dataset = Dataset.from_list(data)
#we are convering our data to a hf friendly dataset for fine-tuning
dataset = dataset.train_test_split(test_size=0.1)
#we are splitting dataset to 90% training and 10% test data
#Dataset is a class, dataset is an instance which we have just initialised

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
#get_peft_model:fn. that wraps base_model with Lora and returns a new model ready for fine-tuning with fewer trainable parameters
#LoraConfig- a class that stores all the settings for Lora fine-tuning(Low Rank Adaptation)
#TaskType- a list of possibilities that tells peft what kind of task we are training for
model_path="/content/LlamaCorn_model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True, #double quantization-quantizing the quantization constants themselves
    bnb_4bit_quant_type = "nf4",  #4 bit quantization format called Normal Float 4 format
    bnb_4bit_compute_dtype = "float16"  #Even though weights are stored in 4 bit, computations will happen in float16
)
#This block will tell transformers "Load the model in 4 bit precision using BitsAndBytes , use double quantization with nf4, and compute in float16"

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    local_files_only = True,
    trust_remote_code = True
)

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
#we are telling the model if there is no padding token(filling with extra tokens until the required length is reached), then we are initialising it as the eos(end of sentence) token => tells the model to stop padding

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = bnb_config,
    device_map = "auto", # mapping the device automatically to the hardware in which it can run ie. cpu/gpu
    local_files_only = True,
    trust_remote_code = True
)

peft_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none"
)
#task_type telling Lora its a CausalLM means its used for predicting the next word in a sequence
#r=8 means the rank of the Lora matrices (8x1 with 1x8 matrix multiplied to create the 8x8 matrix)
#lora_alpha is the scaling factor, it tells how strongly the Lora layers influence the final model
#lora_dropout means randomly dropping 5 percent of the model's connections to prevent overfitting
# we are not adding any bias to the weights

In [ ]:
from transformers import TrainingArguments
#TrainingArguments is a class that stores all the training settings like batch size, learning rate, no. of epochs etc.
training_args = TrainingArguments(
    output_dir="llamacorn_finetuned",
    per_device_train_batch_size = 2,
    #2 training examples are processed at once per gpu - small value beacuse LLMs consume high VRAM
    gradient_accumulation_steps = 4,
    #after processing 2 examples for 4 steps the graient(weight change) is averaged and calculated
    logging_steps = 10,
    #log the training metrics like loss, learning rate every 10 steps
    num_train_epochs = 3,
    # training for full 3 passes over our data
    save_strategy = "epoch",
    # the training will save progress after each epoch
    learning_rate = 2e-4,
    #lr = 0.0002 ie. how much jump the model makes when it learns from its mistakes after each update
    bf16 = False,
    #will not use bfloat16 precision
    fp16 = True,
    # will use floating point 16 precision
    report_to = "none"
    # will not send the logs anywhere
)

In [ ]:
from trl import SFTTrainer
#Supervised Fine Tuning Trainer-this class actually fine tunes the model based on the specified parameters
from datasets import load_dataset

dataset = load_dataset("json", data_files = "/content/drive/MyDrive/LLMCounsellingChatbot/data/finetune_ready_data.json")["train"]
#load_dataset will return a dictionary like object called DatasetDict-this dictionary has splits like "train", "test", "vaildation" etc. so here we are calling the train split from our json dataset

def tokenize(batch):
  conversations = [
      f"User: {row[0]['content']}\nAssistant : {row[1]['content']}"
      for row in batch["messages"]
  ]
  return tokenizer(
      conversations,
      truncation = True,
      #if text is longer than 512 we will cut it(truncate)
      padding = "max_length",
      # if there are shorter texts we will pad them with padding tokens up till 512
      max_length = 512
  )

tokenized_dataset = dataset.map(tokenize, batched = True)
# we are using the map function to map the tokenize function to multiple batches so that computation will be faster, our dataset will be now consisting of tokens instead of raw text in the json format

trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    peft_config = peft_config
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/27 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


Step,Training Loss
10,5.830400


TrainOutput(global_step=12, training_loss=5.002974857886632, metrics={'train_runtime': 26.2039, 'train_samples_per_second': 3.091, 'train_steps_per_second': 0.458, 'total_flos': 257700069900288.0, 'train_loss': 5.002974857886632})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModelForCausalLM, LoraConfig
import torch
import json

base_model_path = "/content/LlamaCorn_model"
lora_adapter_path ="/content/llamacorn_finetuned/checkpoint-12"

with open(f"{lora_adapter_path}/adapter_config.json", "r") as f:
  config_dict = json.load(f)
  #loads the content of checkpoint-12/adapter_config.json as a dictionary

peft_config = LoraConfig(**config_dict)
peft_config.base_model_name_or_path = base_model_path

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True,
    local_files_only=True
)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModelForCausalLM.from_pretrained(
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config = bnb_config,
        device_map = "auto",
        trust_remote_code=True,
        local_files_only=True
    ),
    model_id = lora_adapter_path,
    is_trainable = False,
    # to stop if any further training takes place
    local_files_only = True
)

In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

prompt = "<|im_start|>user Hello, I'm feeling depressed in the classroom and I don't know what to do.<|im_end|>\n<|im_start|>assistant"

output = pipe(
    prompt,
    max_new_tokens = 500,
    temperature = 0.7,
    #controls creativity/ randomness- more means it will be more creative but risk in accurcy
    top_p = 0.9,
    #nucleus sampling-instead of chosing the single most likely word it will chose from a small possible group of words whose combined prob is at least 90%
    do_sample = True,
    #provides new responses each time we run-reduces repetition
    repetition_penalty = 1.1
    #penalty given to reduce repititon
)
print("\n🧠 Assistant:\n", output[0]['generated_text'])

Device set to use cuda:0



🧠 Assistant:
 <|im_start|>user Hello, I'm feeling depressed in the classroom and I don't know what to do.<|im_end|>
<|im_start|>assistant
It sounds like you might be experiencing some feelings of sadness or melancholy in your academic setting. This could stem from a variety of reasons such as:

1. **Feeling overwhelmed with the workload**: If you feel that you have too much homework to complete in a short period of time, this can make you feel stressed and unhappy. It’s also possible if you feel like you’re not learning enough or that you’re not doing well enough on your assignments.
 
2. **Lack of social interaction**: Sometimes, when you’re not feeling happy or fulfilled at school, it may seem like there isn’t anyone around who wants to engage with you. You might feel lonely or disconnected.

3. **Pressure from parents or peers**: Sometimes, people in your life may ask you to do things, or think that you should be doing more, which makes you feel pressured or stressed.

4. **Distrau

In [ ]:
!pip install flask pyngrok

In [ ]:
import os
os.makedirs('templates', exist_ok=True)
print("✅ Templates folder created!")

✅ Templates folder created!


In [ ]:
html_content = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Safe Space - Teen Counselling Assistant</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
        }

        .chat-container {
            width: 90%;
            max-width: 800px;
            height: 90vh;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }

        .chat-header {
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            padding: 20px;
            text-align: center;
            position: relative;
        }

        .chat-header h1 {
            font-size: 24px;
            font-weight: 600;
            margin-bottom: 5px;
        }

        .chat-header p {
            font-size: 14px;
            opacity: 0.9;
        }

        .status-indicator {
            position: absolute;
            top: 15px;
            right: 20px;
            width: 12px;
            height: 12px;
            border-radius: 50%;
            background: #ff4757;
            animation: pulse 2s infinite;
        }

        .status-indicator.online {
            background: #2ed573;
        }

        @keyframes pulse {
            0% { opacity: 1; }
            50% { opacity: 0.5; }
            100% { opacity: 1; }
        }

        .chat-messages {
            flex: 1;
            padding: 20px;
            overflow-y: auto;
            display: flex;
            flex-direction: column;
            gap: 15px;
        }

        .message {
            max-width: 80%;
            padding: 12px 16px;
            border-radius: 18px;
            word-wrap: break-word;
            line-height: 1.4;
            animation: fadeInUp 0.3s ease-out;
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        .message.user {
            background: #4facfe;
            color: white;
            align-self: flex-end;
            margin-left: auto;
        }

        .message.assistant {
            background: #f1f3f4;
            color: #333;
            align-self: flex-start;
            border: 1px solid #e0e0e0;
        }

        .message.system {
            background: #fff3cd;
            color: #856404;
            align-self: center;
            border: 1px solid #ffeaa7;
            max-width: 90%;
            text-align: center;
            font-size: 14px;
        }

        .typing-indicator {
            display: none;
            align-self: flex-start;
            background: #f1f3f4;
            padding: 12px 16px;
            border-radius: 18px;
            border: 1px solid #e0e0e0;
        }

        .typing-dots {
            display: flex;
            gap: 4px;
        }

        .typing-dots span {
            width: 8px;
            height: 8px;
            border-radius: 50%;
            background: #999;
            animation: typing 1.4s infinite;
        }

        .typing-dots span:nth-child(2) { animation-delay: 0.2s; }
        .typing-dots span:nth-child(3) { animation-delay: 0.4s; }

        @keyframes typing {
            0%, 60%, 100% { opacity: 0.3; }
            30% { opacity: 1; }
        }

        .chat-input-container {
            padding: 20px;
            background: #f8f9fa;
            border-top: 1px solid #e0e0e0;
        }

        .chat-input {
            display: flex;
            gap: 12px;
            align-items: flex-end;
        }

        #messageInput {
            flex: 1;
            padding: 12px 16px;
            border: 2px solid #e0e0e0;
            border-radius: 25px;
            font-size: 16px;
            resize: none;
            max-height: 120px;
            min-height: 50px;
            font-family: inherit;
            outline: none;
            transition: border-color 0.3s ease;
        }

        #messageInput:focus {
            border-color: #4facfe;
        }

        #sendButton {
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            border: none;
            width: 50px;
            height: 50px;
            border-radius: 50%;
            cursor: pointer;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 20px;
            transition: transform 0.2s ease, box-shadow 0.2s ease;
            flex-shrink: 0;
        }

        #sendButton:hover {
            transform: scale(1.05);
            box-shadow: 0 4px 12px rgba(79, 172, 254, 0.4);
        }

        #sendButton:disabled {
            background: #ccc;
            cursor: not-allowed;
            transform: none;
            box-shadow: none;
        }

        .disclaimer {
            font-size: 12px;
            color: #666;
            text-align: center;
            padding: 10px 20px;
            background: #f8f9fa;
        }

        .crisis-resources {
            background: #ffe6e6;
            border: 1px solid #ffcccc;
            color: #d63384;
            padding: 15px;
            border-radius: 12px;
            margin: 10px 0;
            font-size: 14px;
            line-height: 1.5;
        }

        /* Responsive design */
        @media (max-width: 768px) {
            .chat-container {
                width: 95%;
                height: 95vh;
                border-radius: 15px;
            }

            .chat-header {
                padding: 15px;
            }

            .chat-header h1 {
                font-size: 20px;
            }

            .message {
                max-width: 85%;
            }

            #messageInput {
                font-size: 16px; /* Prevent zoom on iOS */
            }
        }

        /* Scrollbar styling */
        .chat-messages::-webkit-scrollbar {
            width: 6px;
        }

        .chat-messages::-webkit-scrollbar-track {
            background: #f1f1f1;
            border-radius: 3px;
        }

        .chat-messages::-webkit-scrollbar-thumb {
            background: #c1c1c1;
            border-radius: 3px;
        }

        .chat-messages::-webkit-scrollbar-thumb:hover {
            background: #a8a8a8;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="chat-header">
            <div class="status-indicator" id="statusIndicator"></div>
            <h1>Safe Space</h1>
            <p>Your supportive counselling assistant</p>
        </div>

        <div class="chat-messages" id="chatMessages">
            <div class="message system">
                Welcome! I'm here to listen and support you. This is a safe space to share your thoughts and feelings. How are you doing today?
            </div>
        </div>

        <div class="typing-indicator" id="typingIndicator">
            <div class="typing-dots">
                <span></span>
                <span></span>
                <span></span>
            </div>
        </div>

        <div class="chat-input-container">
            <div class="chat-input">
                <textarea id="messageInput" placeholder="Share what's on your mind..." maxlength="500"></textarea>
                <button id="sendButton" disabled>➤</button>
            </div>
        </div>

        <div class="disclaimer">
            This AI assistant provides supportive conversations but is not a replacement for professional mental health care. If you're in crisis, please contact emergency services or a mental health professional.
        </div>
    </div>

    <script>
        const chatMessages = document.getElementById('chatMessages');
        const messageInput = document.getElementById('messageInput');
        const sendButton = document.getElementById('sendButton');
        const typingIndicator = document.getElementById('typingIndicator');
        const statusIndicator = document.getElementById('statusIndicator');

        let isModelReady = false;

        // Check model status
        async function checkModelStatus() {
            try {
                const response = await fetch('/status');
                const data = await response.json();
                isModelReady = data.model_loaded;

                if (isModelReady) {
                    statusIndicator.classList.add('online');
                    sendButton.disabled = false;
                } else {
                    statusIndicator.classList.remove('online');
                    sendButton.disabled = true;
                    // Check again in 3 seconds
                    setTimeout(checkModelStatus, 3000);
                }
            } catch (error) {
                console.error('Error checking model status:', error);
                setTimeout(checkModelStatus, 5000);
            }
        }

        // Auto-resize textarea
        messageInput.addEventListener('input', function() {
            this.style.height = 'auto';
            this.style.height = Math.min(this.scrollHeight, 120) + 'px';
        });

        // Send message on Enter (but allow Shift+Enter for new lines)
        messageInput.addEventListener('keydown', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                sendMessage();
            }
        });

        // Send button click
        sendButton.addEventListener('click', sendMessage);

        async function sendMessage() {
            const message = messageInput.value.trim();
            if (!message || !isModelReady) return;

            // Add user message
            addMessage(message, 'user');
            messageInput.value = '';
            messageInput.style.height = 'auto';
            sendButton.disabled = true;

            // Show typing indicator
            showTypingIndicator();

            try {
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ message: message })
                });

                const data = await response.json();

                if (data.error) {
                    addMessage(`Sorry, ${data.error}`, 'assistant');
                } else {
                    addMessage(data.response, 'assistant');
                }
            } catch (error) {
                console.error('Error:', error);
                addMessage('I\'m having trouble connecting right now. Please try again in a moment.', 'assistant');
            } finally {
                hideTypingIndicator();
                sendButton.disabled = !isModelReady;
            }
        }

        function addMessage(text, sender) {
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', sender);

            // Check for crisis-related content and format accordingly
            if (text.includes('National Suicide Prevention Lifeline') || text.includes('Crisis Text Line')) {
                messageDiv.innerHTML = `<div class="crisis-resources">${text.replace(/\n/g, '<br>')}</div>`;
            } else {
                messageDiv.textContent = text;
            }

            chatMessages.appendChild(messageDiv);
            scrollToBottom();
        }

        function showTypingIndicator() {
            typingIndicator.style.display = 'block';
            scrollToBottom();
        }

        function hideTypingIndicator() {
            typingIndicator.style.display = 'none';
        }

        function scrollToBottom() {
            setTimeout(() => {
                chatMessages.scrollTop = chatMessages.scrollHeight;
            }, 100);
        }

        // Initialize
        checkModelStatus();
        messageInput.focus();
    </script>
</body>
</html>
'''

with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)
print("✅ HTML file saved in templates folder!")

✅ HTML file saved in templates folder!


In [ ]:
# Step 1: Clear everything and start over
import os
if os.path.exists('app.py'):
    os.remove('app.py')

print("✅ Cleared old files")

✅ Cleared old files


In [ ]:
# Step 2: Create the Flask app with a simple approach
content = """from flask import Flask, render_template, request, jsonify
from transformers import pipeline
import threading
import time

app = Flask(__name__)
pipe = None

def load_model():
    global pipe
    print("Loading model...")
    # Use your existing pipeline code here
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
    from peft import PeftModelForCausalLM, LoraConfig
    import torch
    import json

    base_model_path = "/content/LlamaCorn_model"
    lora_adapter_path = "/content/llamacorn_finetuned/checkpoint-12"

    with open(f"{lora_adapter_path}/adapter_config.json", "r") as f:
        config_dict = json.load(f)

    peft_config = LoraConfig(**config_dict)
    peft_config.base_model_name_or_path = base_model_path

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True, local_files_only=True)
    tokenizer.pad_token = tokenizer.eos_token

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        local_files_only=True
    )

    model = PeftModelForCausalLM.from_pretrained(
        model=base_model,
        model_id=lora_adapter_path,
        is_trainable=False,
        local_files_only=True
    )

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map="auto")
    print("Model loaded!")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    user_message = data.get("message", "")

    if pipe is None:
        return jsonify({"response": "Model still loading..."})

    prompt = f"<|im_start|>user\\n{user_message}<|im_end|>\\n<|im_start|>assistant\\n"

    try:
        output = pipe(prompt, max_new_tokens=200, temperature=0.7, do_sample=True)
        response = output[0]["generated_text"].replace(prompt, "").strip()
        return jsonify({"response": response})
    except:
        return jsonify({"response": "Sorry, I had trouble responding."})

@app.route("/status")
def status():
    return jsonify({"model_loaded": pipe is not None})

if __name__ == "__main__":
    threading.Thread(target=load_model, daemon=True).start()
    app.run(host="0.0.0.0", port=5000, debug=True)
"""

with open('app.py', 'w') as f:
    f.write(content)

print("✅ Flask app created!")

✅ Flask app created!


In [ ]:
from pyngrok import ngrok
import time

# Kill all existing tunnels
ngrok.kill()

# Wait a moment
time.sleep(3)

# Create a completely new tunnel
new_url = ngrok.connect(5000)
print(f"🌐 NEW chatbot URL: {new_url}")
print(f"📝 Copy this URL: {new_url}")

🌐 NEW chatbot URL: NgrokTunnel: "https://7e6183dde16d.ngrok-free.app" -> "http://localhost:5000"
📝 Copy this URL: NgrokTunnel: "https://7e6183dde16d.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# Make sure the app.py file still exists
import os
if os.path.exists('app.py'):
    print("✅ app.py exists")
else:
    print("❌ app.py missing - need to recreate it")

# Restart Flask
print("🔄 Starting Flask...")
exec(open('app.py').read())

✅ app.py exists
🔄 Starting Flask...
Loading model...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
